#### Generating List of sentences

In [5]:
import xml.etree.ElementTree as ET
print('Processing text dataset')

tree = ET.parse("/home/jeet/Academics/CS671/Project/Restaurants_Train.xml")
corpus = tree.getroot()
sentences = [] # List of sentences.
raw_data = corpus.findall('.//sentence')
for sent in raw_data:
    sentences.append(sent.find('text').text)

print ('Generated list of sentences..')
print (sentences[:5])

Processing text dataset
Generated list of sentences..
['But the staff was so horrible to us.', "To be completely fair, the only redeeming factor was the food, which was above average, but couldn't make up for all the other deficiencies of Teodora.", "The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not.", 'Where Gabriela personaly greets you and recommends you what to eat.', "For those that go once and don't enjoy it, all I can say is that they just don't get it."]


#### Creating Test data in unicode format

In [4]:
from keras.preprocessing.text import text_to_word_sequence

test_data = []
count = 1
for sent in sentences:
    word_tokens = text_to_word_sequence(sent, lower=False)
    word_tokens = [w.decode('UTF-8') for w in word_tokens]
    if count < 5:
        print word_tokens
    count+=1
    test_data.append(word_tokens)

[u'But', u'the', u'staff', u'was', u'so', u'horrible', u'to', u'us']
[u'To', u'be', u'completely', u'fair', u'the', u'only', u'redeeming', u'factor', u'was', u'the', u'food', u'which', u'was', u'above', u'average', u'but', u"couldn't", u'make', u'up', u'for', u'all', u'the', u'other', u'deficiencies', u'of', u'Teodora']
[u'The', u'food', u'is', u'uniformly', u'exceptional', u'with', u'a', u'very', u'capable', u'kitchen', u'which', u'will', u'proudly', u'whip', u'up', u'whatever', u'you', u'feel', u'like', u'eating', u'whether', u"it's", u'on', u'the', u'menu', u'or', u'not']
[u'Where', u'Gabriela', u'personaly', u'greets', u'you', u'and', u'recommends', u'you', u'what', u'to', u'eat']


#### Indexing Aspects

In [6]:
import nltk
import numpy as np
test_data = []
train_out= np.zeros(shape=(3044,69))
i=0
for output in raw_data:
    indices = np.zeros(69)
    
    aspectTerms = output.find('aspectTerms')
    if (aspectTerms):
        aspectTerm = aspectTerms.findall('aspectTerm')
        if (aspectTerm):
            for aspect_term in aspectTerm:
                try:
                    indices[s.index(aspect_term.attrib['term'])] = 1
#                     print (indices)
                except:
                    continue
    train_out[i] = indices
    i=i+1

print ("Shape of output tensor:", train_out.shape)

('Shape of output tensor:', (3044, 69))


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: FutureWarning: The behavior of this method will change in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.


#### Defining input data for CRF Training

In [7]:
train_input = []
i = 0
for sent in sentences:
    tuple_sent = []
    word_token = text_to_word_sequence(sent, lower=False)
    j = 0
    for word in word_token:
        input_tuple = ()
        input_tuple = input_tuple + (word.decode('UTF-8'),)
        if train_out[i][j] == 0:
            input_tuple = input_tuple + ('NA',)
        else:
            input_tuple = input_tuple + ('A',)
        tuple_sent.append(input_tuple)
        j=j+1
    train_input.append(tuple_sent)
    i=i+1

#### Defining training and testing data

In [8]:
train_data = train_input[:2739]
test = test_data[2739:]
true_test = train_input[2739:]

In [9]:
from nltk.tag import CRFTagger
ct = CRFTagger()

ct.train(train_data,'model_Crf_tagger')

In [11]:
ct.tag_sents(test)

[]

In [12]:
ct.evaluate(true_test)

1.0